In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
import numpy as np

Сохраним настройки

In [2]:
models = [['my_model_(drift_master).h5', 'my_model_(loosey-goosey).h5', 
          'my_model_(little_less_loosey-goosey).h5'], [0.4,0.6,0.6], [0.3,0.4,0.4]]

In [3]:
import pandas as pd

settings = pd.DataFrame(data=np.array(models).T, 
                        columns=['name', 'speed_limit', 'break_power'])

settings

,name,speed_limit,break_power
0,my_model_(drift_master).h5,0.4,0.3
1,my_model_(loosey-goosey).h5,0.6,0.4
2,my_model_(little_less_loosey-goosey).h5,0.6,0.4


Выберем модель

In [4]:
model_number = 0 #choose your model

Загрузим настройки

In [5]:
model_name = settings.iloc[model_number].loc['name']
speed_limit = float(settings.iloc[model_number].loc['speed_limit'])
break_power = float(settings.iloc[model_number].loc['break_power'])

Определим пространство действий

In [6]:
Actions = np.array([np.array([0, speed_limit, 0]),
np.array([0, 0, 0]),
np.array([0, 0, 0.4]),
np.array([-1, 0, 0]),
np.array([1, 0, 0]),
np.array([-1, 0, break_power]),
np.array([1, 0, break_power])])

Загрузим модель

In [7]:
model = load_model(model_name)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0     

In [9]:
import gym
env = gym.make('CarRacing-v0')  #создадим окружение gym
state = env.reset()             #получим изначальное состояние
states = state.reshape(1,96,96,3) #reshape для нейронки
# for i in range(5):
#     states[:,i] = state
    
for _ in range(50):            #толкнём машинку на старте
    action = np.array([0,1,0])
    next_state, reward, done, _ = env.step(action) # произведём толчок, получим следующее состояние мира
    env.render()                                   # отрендерим кадр
    states = next_state.reshape(1,96,96,3)         # reshape для нейронки
for _ in range(4000):
    action_dist = model.predict(states)            # предскажем следующее действие на основе текущего состояния
    action_num = np.argmax(action_dist)            # выберем наиболее вероятное
    action = Actions[action_num]                   # 
    next_state, reward, done, _ = env.step(action) # совершим действие, получим следующее состояние мира
    env.render()                                   # отрендерим кадр
    states = next_state.reshape(1,96,96,3)         # получим следующее состояние
    if done:                                       #
        break                                      # если всё проехали, заканчиваем цикл
env.close()                                        # закрываем окружение

Track generation: 1130..1417 -> 287-tiles track


D:\Anaconda\envs\CarRace\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
